# Battle of the neighborhoods final assignment

# introduction

Many people enjoy good food. Especially fast food that is healthy and cheap. Eating pizza with your friends can be an enjoyment and an easy method to ease your hunger. Not only can we get pizza but we can also get various other consumables such as: soft drinks, cookies and deserts. Pizza stores are multi-purpose restaurants. It can be used for takeaways or eat your meal inside the store. The consumption of fast food has increased dramatically in the last few years. Franchises are expanding business and opening multiple new stores in a lot of location within Amsterdam The Netherlands. As a result of this, there are many different kinds of pizza places in Amsterdam. sole proprietorships as well as big company franchise stores. Opening new pizza stores comes with its own risks and gains. Ass there are many pizza places located in Amsterdam, what area is the best to open one? 

# business problem

Research question is: in what area of Amsterdam are the least amount of pizza places and is it economically feasible to start a pizza place in a the area where there are the least amount of pizza places? Based on the outcome of the capstone data science project using data science methodology and machine learning techniques. By using these machine learning abilities, are we able to give concrete information about the best place to start a new pizza place

# Data

Based on what we want to know to solve this business case, there are some factors that will influence our outcome and predictions.
1.	the amount of pizza places in Amsterdam and in their separate clusters.
2.	The longitude and the latitude of the neighbourhoods where these pizza places are located in.
3.	And the venue data that we will use to cluster the data.
We will use Wikipedia to extract data from their website. We will use their neighbourhoods in Amsterdam data. There are a total of 107 neighbourhoods in Amsterdam. we will be using webscraping to extract the data from their website. Furthermore, we will be using pandas to transform the data into dataframes that we can use for our clustering. We will use the foursquare API to gather the coordinates and venue data that will be relevant for our clustering. 


In [2]:
# these are all the libraries/packages that we will use for this assignment

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json
from geopy.geocoders import Nominatim
import requests 
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

Libraries imported.


In [4]:
import geocoder

In [6]:
# getting url for assignment
data_set = requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_of_Amsterdam').text

In [8]:
# parsing data
soup = BeautifulSoup(data_set, 'html.parser')

In [9]:
# create empty list to fill data in
neighborhoodList = []

In [10]:
# filling data into empty list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [12]:
# transform list into pd dataframa
NL_df = pd.DataFrame({"Neighborhood": neighborhoodList})

NL_df.head()

,Neighborhood
0,Template:Neighborhoods of Amsterdam
1,Admiralenbuurt
2,Amsteldorp
3,Amsterdam Oud-West
4,Amsterdam Oud-Zuid


In [14]:
NL_df.shape

(107, 1)

In [16]:
# defining function to gather coordinates of the neigborhoods with loop until we get all coordinates
def get_latlng(neighborhood):
    
    lat_lng_coords = None
   
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Amsterdam, The Netherlands'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [18]:
# making a seperate dataframe for the coordinates
coords = [ get_latlng(neighborhood) for neighborhood in NL_df["Neighborhood"].tolist() ]

In [20]:
#appending dataframe with new columns
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [22]:
# replacing existing dataframe columns with the columns with data in it.
NL_df['Latitude'] = df_coords['Latitude']
NL_df['Longitude'] = df_coords['Longitude']

In [23]:
print(NL_df.shape)
NL_df

(107, 3)


,Neighborhood,Latitude,Longitude
0,Template:Neighborhoods of Amsterdam,52.401270,4.936110
1,Admiralenbuurt,52.372752,4.856359
2,Amsteldorp,52.339680,4.918740
3,Amsterdam Oud-West,52.365390,4.870220
4,Amsterdam Oud-Zuid,52.352350,4.877880
5,Amsterdam Science Park,52.354300,4.958010
6,Apollobuurt,52.350294,4.867990
7,Betondorp,52.423405,4.833395
8,Bijlmermeer,52.307031,4.969744
9,Binnenstad (Amsterdam),52.369930,4.907880


In [25]:
# dataframe into csv
NL_df.to_csv("NL_df.csv", index=False)

In [29]:
# get the coordinates of Amsterdam
address = 'Amsterdam, The Netherlands'

geolocator = Nominatim(user_agent="aniket88@live.nl")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Amsterdam, The Netherlands {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Amsterdam, The Netherlands 52.3727598, 4.8936041.


In [32]:
# plotting map
map_NL = folium.Map(location=[latitude, longitude], zoom_start=11)

# add dots to map
for lat, lng, neighborhood in zip(NL_df['Latitude'], NL_df['Longitude'], NL_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_NL)  

In [34]:
map_NL

In [35]:
# saving map into html file
map_NL.save('map_NL.html')

In [37]:
CLIENT_ID = '4NN3HELE3VZE314P0DJAKPHDZ5PERFUXQCJCJ3BEF2WJMKEE' # your Foursquare ID
CLIENT_SECRET = 'RSMDGI4LFG1JUEDLSOM2QE2K3ONLSG0JASTBKKPYRH1GS3DK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4NN3HELE3VZE314P0DJAKPHDZ5PERFUXQCJCJ3BEF2WJMKEE
CLIENT_SECRET:RSMDGI4LFG1JUEDLSOM2QE2K3ONLSG0JASTBKKPYRH1GS3DK


In [39]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(NL_df['Latitude'], NL_df['Longitude'], NL_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [40]:
venues_df = pd.DataFrame(venues)

# naming the columns
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(9311, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Template:Neighborhoods of Amsterdam,52.40127,4.93611,Patisserie Kwekkeboom BovenIJ,52.397824,4.937422,Bakery
1,Template:Neighborhoods of Amsterdam,52.40127,4.93611,Café 't Sluisje,52.391134,4.938363,Bar
2,Template:Neighborhoods of Amsterdam,52.40127,4.93611,Action,52.399487,4.936319,Discount Store
3,Template:Neighborhoods of Amsterdam,52.40127,4.93611,Noorderparkbad,52.398171,4.918952,Pool
4,Template:Neighborhoods of Amsterdam,52.40127,4.93611,BudgetCam.nl,52.386930,4.930856,Camera Store


In [41]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Admiralenbuurt,100,100,100,100,100,100
Amsteldorp,100,100,100,100,100,100
Amsterdam Oud-West,100,100,100,100,100,100
Amsterdam Oud-Zuid,100,100,100,100,100,100
Amsterdam Science Park,91,91,91,91,91,91
Apollobuurt,100,100,100,100,100,100
Betondorp,53,53,53,53,53,53
Bijlmermeer,97,97,97,97,97,97
Binnenstad (Amsterdam),100,100,100,100,100,100


In [42]:
# how many unique categories do we have?
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 283 uniques categories.


In [81]:
# names of the unique categories
venues_df['VenueCategory'].unique()[:1000000]

array(['Bakery', 'Bar', 'Discount Store', 'Pool', 'Camera Store',
       'Ice Cream Shop', 'Drugstore', 'Golf Course', 'Café',
       'Mexican Restaurant', 'Skate Park', 'Sandwich Place',
       'Gym / Fitness Center', 'Supermarket', 'Asian Restaurant',
       'Turkish Restaurant', 'French Restaurant', 'Park', 'Restaurant',
       'Thai Restaurant', 'Caribbean Restaurant', 'Campground', 'Market',
       'Music Venue', 'Bus Stop', 'Coffee Shop', 'Hotel', 'Snack Place',
       'Church', 'Athletics & Sports', 'Indonesian Restaurant',
       'Soccer Field', 'Farm', 'Dance Studio', 'Boat or Ferry',
       'Rock Club', 'Sports Club', 'Pub', 'Spa', 'Ramen Restaurant',
       'Falafel Restaurant', 'Tea Room', 'Deli / Bodega',
       'Lebanese Restaurant', 'Scandinavian Restaurant', 'Pizza Place',
       'Greek Restaurant', 'Pet Café', 'Fried Chicken Joint', 'Wine Bar',
       'Gym', 'Hostel', 'Yoga Studio', 'Italian Restaurant',
       'Indian Restaurant', 'Movie Theater', 'Bookstore', 'Beer G

In [44]:
# are their unique neigborhoods in our dataset?
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [45]:

NL_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NL_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [NL_onehot.columns[-1]] + list(NL_onehot.columns[:-1])
NL_onehot = NL_onehot[fixed_columns]

print(NL_onehot.shape)
NL_onehot.head()

(9311, 284)


,Neighborhoods,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Bowling Alley,Brasserie,Breakfast Spot,Brewery,Bridal Shop,Bridge,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Camera Store,Campaign Office,Campground,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dog Run,Doner Restaurant,Drugstore,Dutch Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Friterie,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Neighborhood,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Resort,Rest Area,Restaurant,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Tour Provider,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,VR Cafe,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Template:Neighborhoods of Amsterdam,0,0,0,0,0,0,0,0,0

In [47]:
# restting index and filtering column using the mean for further evaluations
NL_grouped = NL_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(NL_grouped.shape)
NL_grouped

(107, 284)


,Neighborhoods,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Bowling Alley,Brasserie,Breakfast Spot,Brewery,Bridal Shop,Bridge,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Camera Store,Campaign Office,Campground,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dog Run,Doner Restaurant,Drugstore,Dutch Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Friterie,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Neighborhood,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Resort,Rest Area,Restaurant,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Tour Provider,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,VR Cafe,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Admiralenbuurt,0.00,0.00,0.00,0.000000,0.000000,0.000

In [52]:
# how many pizza places are their?
len(NL_grouped[NL_grouped["Pizza Place"] > 0])

79

In [53]:
NL_pizza = NL_grouped[["Neighborhoods","Pizza Place"]]

In [55]:
NL_pizza.head()

,Neighborhoods,Pizza Place
0,Admiralenbuurt,0.03
1,Amsteldorp,0.02
2,Amsterdam Oud-West,0.02
3,Amsterdam Oud-Zuid,0.01
4,Amsterdam Science Park,0.00


# methodology

in step one we gathered the information from the website: wickipedia and extracted the information from wikipedia by using a webscraper. 

in step two we used the goecoder python package to gather the longitude and latitude of these neigborhoods in amsterdam so we were abel to use the foursquare API 

in step three we imported the data into a pandas dataframe for further cleaning and analysis purposes. 

in step four we used folium to plot the map of amsterdam and to see if the coordinates were correclty added to the map.

in step five 

In [59]:
# we use 3 clusters
kclusters = 3

NL_clustering = NL_pizza.drop(["Neighborhoods"], 1)


kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NL_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 0, 0, 2, 2, 1, 0, 0, 1])

In [62]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
NL_merged = NL_pizza.copy()

# add clustering labels
NL_merged["Cluster Labels"] = kmeans.labels_

In [63]:
# merge the columns
NL_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
NL_merged.head()

,Neighborhood,Pizza Place,Cluster Labels
0,Admiralenbuurt,0.03,1
1,Amsteldorp,0.02,0
2,Amsterdam Oud-West,0.02,0
3,Amsterdam Oud-Zuid,0.01,0
4,Amsterdam Science Park,0.00,2


In [70]:
# merging to add longitude and latitude into existing dataframe
NL_merged = NL_merged.join(NL_df.set_index("Neighborhood"), on="Neighborhood")

#print(NL_merged.shape)
NL_merged.head() # check the last columns!

,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
53,Negen Straatjes,0.02,0,52.37047,4.88324
62,Oosterparkbuurt (Amsterdam),0.01,0,52.35947,4.92298
61,Oosterdokseiland,0.01,0,52.36993,4.90788
60,Oostelijke Eilanden,0.01,0,52.36852,4.92872
58,Olympisch Kwartier,0.01,0,52.36993,4.90788


In [71]:
# sort the results by Cluster Labels
print(NL_merged.shape)
NL_merged.sort_values(["Cluster Labels"], inplace=True)
NL_merged

(107, 5)


,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
53,Negen Straatjes,0.020000,0,52.370470,4.883240
72,Overtoomse Veld,0.010000,0,52.363590,4.836800
71,Overtoombuurt,0.020000,0,52.359110,4.861340
70,Overhoeks,0.010000,0,52.391890,4.906040
40,Jordaan,0.020000,0,52.369520,4.880910
38,Java-eiland,0.010000,0,52.375140,4.895050
106,Zuidas,0.010000,0,52.336690,4.875920
14,Burgwallen Nieuwe Zijde,0.010000,0,52.376211,4.895015
15,Burgwallen Oude Zijde,0.010000,0,52.369930,4.907880
18,Czaar Peterbuurt,0.010000,0,52.369786,4.929039


In [73]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add pointers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NL_merged['Latitude'], NL_merged['Longitude'], NL_merged['Neighborhood'], NL_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [75]:
# saving map as html 
map_clusters.save('map_clusters.html')

In [76]:
# locating the neighborhoods in cluster 0
NL_merged.loc[NL_merged['Cluster Labels'] == 0]

,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
53,Negen Straatjes,0.020000,0,52.370470,4.883240
72,Overtoomse Veld,0.010000,0,52.363590,4.836800
71,Overtoombuurt,0.020000,0,52.359110,4.861340
70,Overhoeks,0.010000,0,52.391890,4.906040
40,Jordaan,0.020000,0,52.369520,4.880910
38,Java-eiland,0.010000,0,52.375140,4.895050
106,Zuidas,0.010000,0,52.336690,4.875920
14,Burgwallen Nieuwe Zijde,0.010000,0,52.376211,4.895015
15,Burgwallen Oude Zijde,0.010000,0,52.369930,4.907880
18,Czaar Peterbuurt,0.010000,0,52.369786,4.929039


In [77]:
# locating neigborhoods in cluster 1
NL_merged.loc[NL_merged['Cluster Labels'] == 1]

,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
29,Gaasperdam,0.026316,1,52.296770,4.978260
50,Molenwijk (Amsterdam),0.027778,1,52.421012,4.890155
59,Omval,0.030000,1,52.344031,4.914592
35,Houthaven,0.040000,1,52.394127,4.877061
0,Admiralenbuurt,0.030000,1,52.372752,4.856359
26,Eendracht (Amsterdam),0.033333,1,52.382060,4.784360
44,Kinkerbuurt,0.030000,1,52.364695,4.859795
45,Kolenkit District,0.030000,1,52.377660,4.842810
33,Haarlemmerbuurt (Amsterdam),0.030000,1,52.384964,4.882090
52,NDSM,0.030000,1,52.400090,4.897220


In [78]:
# locating neigborhoods in cluster 2
NL_merged.loc[NL_merged['Cluster Labels'] == 2]

,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
25,Eastern Docklands,0.0,2,52.387016,4.905738
81,Schinkelbuurt,0.0,2,52.351657,4.852859
80,Schellingwoude,0.0,2,52.384190,4.963850
79,Ruigoord,0.0,2,52.410000,4.749252
56,Nieuwendam,0.0,2,52.388105,4.968095
12,Buitenveldert,0.0,2,52.330930,4.875180
75,Ransdorp,0.0,2,52.393060,4.994260
11,Buikslotermeer,0.0,2,52.396250,4.935790
83,Slotervaart (neighbourhood),0.0,2,52.352360,4.832320
22,De Wallen,0.0,2,52.371690,4.897240
